# Imports

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt

# $L_2$ Regularized Logistic Regression

## Sigmoid Function
**Arguments:**

* **`z`** : 
  * A 2D numpy array of floats

**Returns:**

* Sigmoid value of **z** with the same shape as **z**
<br><br>$\hspace{20mm}{\sigma(z)} = \frac{1}{1+e^{-z}}$


In [ ]:
def compute_sigmoid(z):
    sigmoid_value = 1/(1 + np.exp(-z)) # broadcasting
    return sigmoid_value

## Hypothesis Function in Logistic Regression
**Arguments:**

* **`X`** : Design Matrix
  * A 2D numpy array of shape (num of instances, num of features)

* **`w`** : Parameters corresponding to each feature
  * A 2D numpy array of shape (num of features, 1)

* **`b`** :  Intercept value
  * A float value


**Returns:**

* Hypothesis value for the given data
 * A 2D numpy array of shape (num of instances, 1) <br><br>$\hspace{20mm}H = \sigma(Xw+b)\\[0.1pt]$
<br>$\hspace{2cm}$(where $\sigma$ represents the sigmoid function) 

In [ ]:
def compute_hypothesis(X, w, b):
    z = np.dot(X, w) + b #broadcasting
    H = compute_sigmoid(z)
    return H

## $L_2$ Regularized Cost Function

**Arguments:**

* **`X`** : Design Matrix
  * A 2D numpy array of shape (num of instances, num of features)

* **`Y`** : Target values corresponding to each training instance in $X$
  * A 2D numpy array of shape (num of instances, 1)

* **`w`** : Parameters corresponding to each feature
  * A 2D numpy array of shape (num of features, 1)

* **`b`** :  Intercept value
  * A float value

* **`Lambda`** :  Regularization parameter($\lambda$)
  * A float value


**Returns:**

* $L_2$ Regularized cost value for the given data <br><br>$\hspace{20mm}J_{w,b}(X)=\frac{-1}{m}\left [Y^Tlog(H)+(1-Y)^Tlog(1-H) \right ]+ \frac{\lambda}{2m}w^Tw\\[0.1pt]$
<br>$\hspace{2cm}$(where $H$ is the Hypothesis value of $X$, and $m$ is the number of instances) 

In [ ]:
def compute_L2_cost(X, Y, w, b, Lambda):
    H = compute_hypothesis(X, w, b)
    m = X.shape[0]
    log_loss = -1/m *(np.dot(Y.T, np.log(H)) + np.dot((1-Y).T, np.log(1-H)))
    regularization_term = (Lambda/(2*m))*np.dot(w.T, w)
    cost_value =  log_loss + regularization_term
    return cost_value.squeeze()

## Gradients of $L_2$ Regularized Cost Function

**Arguments:**

* **`X`** : Design Matrix
  * A 2D numpy array of shape (num of instances, num of features)

* **`Y`** : Target values corresponding to each training instance in $X$
  * A 2D numpy array of shape (num of instances, 1)

* **`w`** : Parameters corresponding to each feature
  * A 2D numpy array of shape (num of features, 1)

* **`b`** :  Intercept value
  * A float value

* **`Lambda`** :  Regularization parameter($\lambda$)
  * A float value



**Returns:**
* Gradient of the cost function corresponding to the parameters of each feature.
 * A 2D numpy array of shape (num of features, 1)<br><br>$\hspace{20mm}\frac{dJ}{dw} = \frac{1}{m}\left [ X^T(H-Y) + \lambda w\right ]$<br><br>
* Gradient corresponding to the intercept value
 * A float value<br><br>$\hspace{20mm}\frac{dJ}{db} = \frac{1}{m}\sum (H-Y) \\[0.1pt]  \\[0.1pt]$
<br>$\hspace{2cm}$(where $H$ is the Hypothesis value of $X$, and $m$ is the number of instances) 


In [ ]:
def gradient_of_L2_cost(X, Y, w, b, Lambda):
    H = compute_hypothesis(X, w, b)
    m = X.shape[0]
    dw = (1.0/m)*(np.dot(X.T, H - Y) + Lambda*w)
    db = (1.0/m)*np.sum(H - Y)
    return dw, db

## Gradient Descent in $L_2$ Regularized Logistic Regression

**Arguments:**

* **`X`** : Design Matrix
  * A 2D numpy array of shape (num of instances, num of features)

* **`Y`** : Target values corresponding to each training instance in $X$
  * A 2D numpy array of shape (num of instances, 1)

* **`w`** : Initial parameters corresponding to each feature
  * A 2D numpy array of shape (num of features, 1)

* **`b`** :  Initial intercept value
  * A float value

* **`cost_diff_threshold`** : threshold value for the absolute cost difference to stop iterating in gradient descent (*Convergence Criteria*)
  * A float value

* **`learning_rate`** :  Learning rate($\alpha$)
  * A float value

* **`Lambda`** :  Regularization parameter($\lambda$)
  * A float value

**Returns:**
* `w`: Optimal parameters of features($w$'s)
 * A 2D numpy array with the same shape as the argument $w$<br>
$\hspace{10mm}w = w - \alpha \frac{dJ}{dw}$<br><br>
* `b`: Optimal intercept value<br>
$\hspace{20mm}b = b - \alpha \frac{dJ}{db}$<br><br>

**NOTE:**
* The gradient descent is said to be converged when the absolute value of the cost difference is less than the given threshold.
* Stop iterating when the gradient descent starts to diverge.

In [ ]:
def gradient_descent(X, Y, w, b, cost_diff_threshold, learning_rate, Lambda):
    prev_cost = compute_L2_cost(X, Y, w, b, Lambda)
    cost_diff = cost_diff_threshold + 1
    i = 0

    while(abs(cost_diff) > cost_diff_threshold):
        dw, db = gradient_of_L2_cost(X, Y, w, b, Lambda)
        w = w - (learning_rate * dw)
        b = b - (learning_rate * db)

        cost = compute_L2_cost(X, Y, w, b, Lambda)
        cost_diff = cost - prev_cost
        if(cost_diff > 0): # diverging
            break
        prev_cost = cost 
        i+=1
            
    return w, b

## One-vs-Rest for Multi-Class Classification


**Arguments:**

* **`X`** : Design Matrix
  * A 2D numpy array of shape (num of instances, num of features)

* **`Y`** : Target values corresponding to each training instance in $X$
  * A 2D numpy array of shape (num of instances, 1)

* **`w`** : Initial parameters corresponding to each feature
  * A 2D numpy array of shape (num of features, 1)

* **`b`** :  Initial intercept value
  * A float value

* **`cost_diff_threshold`** : threshold value for the absolute cost difference to stop iterating in gradient descent (*Convergence Criteria*)
  * A float value

* **`learning_rate`** :  Learning rate($\alpha$)
  * A float value

* **`Lambda`** :  Regularization parameter($\lambda$)
  * A float value



**Returns:**
* `classwise_params_dict`
 * A dict where the keys are the class labels and the values are the respective optimal parameters [$w$, $b$]<br>
   * where $w$ is a 2D numpy array with the same shape as the argument $w$<br>
   * $b$ is the optimal value of the intercept (float)


In [ ]:
def one_vs_rest(X, Y, w, b, cost_diff_threshold, learning_rate, Lambda):
    classes = np.unique(Y)
    classes.sort()

    classwise_params_dict = {}
    for class_label in classes:
        class_Y = np.where(Y == class_label, 1, 0)
        optimal_w, optimal_b = gradient_descent(X, class_Y, w, b, cost_diff_threshold, learning_rate, Lambda)
        classwise_params_dict[class_label] = [optimal_w, optimal_b]

    return classwise_params_dict

## Prediction in One-vs-Rest Approach 


**Arguments:**

* **`X`** : Design Matrix to predict the class labels for
  * A 2D numpy array of shape (num of instances, num of features)

* **`classwise_params_dict`**: A dict where the keys are the class labels(ints) and the values are the respective optimal parameters [$w$, $b$]<br>
   * where $w$ is a 2D numpy array with the shape (num of features, 1)<br>
   * $b$ is the optimal value of the intercept (float)



**Returns:**

* Predicted class labels of $X$
 * A 2D numpy array of shape (num of instances, 1)
 * If there is a tie among the class probabilities, predict the class label with the smallest value among the tied classes.

In [ ]:
def predict_labels_in_one_vs_rest(X, classwise_params_dict):
    classes = sorted(classwise_params_dict.keys())
    classes = np.array(classes)
    m = X.shape[0] 
    classwise_hypothesis = np.zeros((len(classes), m))

    for idx, class_label in enumerate(classes):
        params = classwise_params_dict[class_label]
        w = params[0]
        b = params[1]
        classwise_hypothesis[idx] = compute_hypothesis(X, w, b).ravel()
        
    predicted_class_indices = np.argmax(classwise_hypothesis, axis=0).ravel()
    predictions = classes[predicted_class_indices]
    return predictions.reshape(-1,1)

# Iris Flower Species Prediction

## Data Collection

Features

In [ ]:
data = load_iris()
df = pd.DataFrame(data.data, columns=data.feature_names)
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


Target Feature

In [ ]:
print(data.target_names)
df['species'] = data.target_names[data.target]
df

['setosa' 'versicolor' 'virginica']


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


## Data Preprocessing

### Checking for missing values

In [ ]:
df.isna().sum()

sepal length (cm)    0
sepal width (cm)     0
petal length (cm)    0
petal width (cm)     0
species              0
dtype: int64

### Handling duplicates

In [ ]:
df[df.duplicated(keep=False)]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
101,5.8,2.7,5.1,1.9,virginica
142,5.8,2.7,5.1,1.9,virginica


In [ ]:
df = df.drop_duplicates()

Converting data to numpy arrays

In [ ]:
df = df.replace({'setosa': 0, 'versicolor': 1, 'virginica':2})
target_Y = df.pop('species')

iris_X = df.to_numpy()
iris_Y = target_Y.to_numpy().reshape(-1,1)

## Applying Logistic Regression for Iris Flower Species Prediction

### Splitting data into Train, Validation and Test sets

In [ ]:
def split_train_val_test(X, Y):
    np.random.seed(11235810) 
    data_count = X.shape[0]
    permuted_indices = np.random.permutation(data_count)

    train_ratio = 0.6
    validation_ratio = 0.2
    train_count = np.int(train_ratio * data_count)
    val_count = np.int(validation_ratio * data_count)

    shuffled_X = X[permuted_indices]
    shuffled_Y = Y[permuted_indices]

    train_X = shuffled_X[:train_count]
    train_Y = shuffled_Y[:train_count]

    val_X = shuffled_X[train_count:train_count+val_count]
    val_Y = shuffled_Y[train_count:train_count+val_count]
    
    test_X = shuffled_X[train_count+val_count:]
    test_Y = shuffled_Y[train_count+val_count:]

    return train_X, train_Y, val_X, val_Y, test_X, test_Y

In [ ]:
train_X, train_Y, val_X, val_Y, test_X, test_Y = split_train_val_test(iris_X, iris_Y)

### Training

 Parameter & Hyperparameter values

In [ ]:
n = train_X.shape[1]
w = np.zeros((n,1))
b = 0
cost_diff_threshold = 1e-7
learning_rate = 0.001
Lambda = 0

Using One-vs-Rest Approach to Train

In [ ]:
classwise_params_dict = one_vs_rest(train_X, train_Y, w, b, cost_diff_threshold, learning_rate, Lambda)

### Prediction & Evaluation

In [ ]:
def predict_and_evaluate(X, Y, classwise_params_dict):
    pred_Y = predict_labels_in_one_vs_rest(X, classwise_params_dict)
    correct_predictions = np.count_nonzero(pred_Y.ravel() == Y.ravel())
    accuracy = correct_predictions/len(Y)
    return accuracy   

In [ ]:
accuracy = predict_and_evaluate(val_X, val_Y, classwise_params_dict)
print("Accuracy on Validation Data: ", accuracy)

Accuracy on Validation Data:  1.0


We got 100% accuracy on the validation set without regularization ($\lambda=0$). 

Since there are only 4 features in the dataset, we don't need to use regularization here.

### Training on the entire Train Data & Reporting the Accuracy on the Test Data

In [ ]:
# Merging train & validation sets
all_train_X = np.vstack((train_X, val_X))
all_train_Y = np.vstack((train_Y, val_Y))

# Training
classwise_params_dict = one_vs_rest(all_train_X, all_train_Y, w, b, cost_diff_threshold, learning_rate, Lambda)

# Evaluation
accuracy = predict_and_evaluate(test_X, test_Y, classwise_params_dict)
print("Accuracy on Test Data: ", accuracy)

Accuracy on Test Data:  0.967741935483871
